In [117]:
!pip install -q langchain
!pip install -q langchain-community
!pip install -q langchain-core
!pip install -q llama-index
!pip install -q llama-index-embeddings-ollama
!pip install -q chromadb

In [13]:
import os
# os.environ['OPENAI_API_KEY'] = os.environ.get('OPENAI_API_KEY')

In [19]:
from llama_index.embeddings.ollama import OllamaEmbedding

In [65]:
embed_model1 = OllamaEmbedding(
    model_name='llama2',
    base_url='http://localhost:11434',
    ollama_additional_kwargs={'mirostat': 0},
)

In [66]:
embed_model1.get_text_embedding_batch(
    ["This is an example!", "This is another example"], show_progress=True
)
q1 = embed_model1.get_query_embedding("Why is the sky blue?")
len(q1)

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

4096

In [67]:
embed_model2 = OllamaEmbedding(
    model_name='nomic-embed-text',
    base_url='http://localhost:11434',
    ollama_additional_kwargs={'mirostat': 0},
)

In [58]:
embed_model2.get_text_embedding_batch(
    ["This is an example!", "This is another example"], show_progress=True
)
q2 = embed_model2.get_query_embedding("Why is the sky blue?")
len(q2)

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

768

In [68]:
embed_model3 = OllamaEmbedding(
    model_name='mxbai-embed-large',
    base_url='http://localhost:11434',
    ollama_additional_kwargs={'mirostat': 0},
)

In [69]:
embed_model3.get_text_embedding_batch(
    ["This is an example!", "This is another example"], show_progress=True
)
q3 = embed_model3.get_query_embedding("Why is the sky blue?")
len(q3)

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

1024

In [71]:
embed_model4 = OllamaEmbedding(
    model_name='stable-code',
    base_url='http://localhost:11434',
    ollama_additional_kwargs={'mirostat': 0},
)

In [70]:
# embed_model4.get_text_embedding_batch(
#     ["This is an example!", "This is another example"], show_progress=True
# )
# q4 = embed_model4.get_query_embedding("Why is the sky blue?")
# len(q4)

In [118]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import CharacterTextSplitter

In [97]:
model = ChatOllama(model="llama3")

In [105]:
urls = [
    "https://adventofcode.com/2015/day/1",
    "https://adventofcode.com/2022/day/11",
    "https://adventofcode.com/2015/day/7",
    "https://adventofcode.com/2023/day/21",
]
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

In [109]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1500, chunk_overlap=100)
doc_splits = text_splitter.split_documents(docs_list)

In [119]:
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-advent",
    embedding=embeddings.OllamaEmbeddings(model="nomic-embed-text"),
)

In [120]:
retriever = vectorstore.as_retriever()

In [123]:
print("Before RAG")
before_rag_template = "What is Advent of Code day {day} task about?"
before_rag_propmt_template = ChatPromptTemplate.from_template(before_rag_template)
before_rag_chain = before_rag_propmt_template | model | StrOutputParser()
print(before_rag_chain.invoke({"day": 11}))

Before RAG
A great question!

Advent of Code Day 11: "Spin Metal" (2021) is a programming puzzle that involves analyzing and manipulating metal spinning patterns.

In this problem, you're given a series of instructions to spin metal into different shapes. The goal is to determine the total number of units of each type of metal that will be produced after following these instructions.

The input data consists of a sequence of lines, where each line represents an instruction to spin metal. Each line starts with a letter indicating the type of metal (A, B, or C), followed by the number of units to produce, and then either "add" or "remove" depending on whether you need to add or remove units from the current production.

Your task is to write a program that can parse these instructions and calculate the total number of each type of metal produced. The solution involves using a simple counter approach and some basic arithmetic operations.

If you're interested in learning more, I encourage

In [124]:
print("After RAG")
rag_template = """Here is some context that might help you answer the question:
\n\n{context}\n\n
Question: {question}
"""
rag_propmt_template = ChatPromptTemplate.from_template(rag_template)
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | (rag_propmt_template | model | StrOutputParser())
print(before_rag_chain.invoke("What is Advent of Code day 11} task about?"))

After RAG
Advent of Code is an annual Christmas-themed programming challenge that was created by Eric Meyer, a computer scientist and author. The challenge consists of 25 small programming puzzles, released one per day from December 1st to December 25th.

Day 11 is one of the tasks in Advent of Code. Here's a brief summary:

**Day 11:**

Title: "Monkey in the Middle"

You're given a set of monkey notes that describe how each monkey moves around and interacts with the other monkeys. Each monkey has its own rules for moving items (called "items" or "worry levels") into their home trees.

The goal is to simulate the monkeys' behavior over several rounds, keeping track of the total worry level in all the monkeys' home trees. The worry level is calculated based on the sum of the item values and a set of simple arithmetic operations (addition, multiplication, squaring).

The task requires you to write a program that takes the monkey notes as input, simulates the monkeys' behavior, and return